In [23]:
import os
import shutil
import glob
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import box
import numpy as np
import rasterio
from rasterio.features import geometry_mask
import sys
from gdal_retile import main as gdal_retile_main
import pandas as pd

In [2]:
# Project params

project_path = r"C:\Users\owyn\Documents\SURV509\coastal classification"
input_dir = os.path.join(project_path, "inputs")

### Merging geojpeg files in each region

In [13]:
# Define paths
aerial_imagery_dir = os.path.join(input_dir, '0.3m Aerial Imagery')
urban_shapefile = os.path.join(input_dir, 'urban', 'urban-areas.shp')
coastline_shapefile = os.path.join(input_dir, 'linz', 'lds-nz-coastlines-topo-150k-JPEG-SHP', 'nz-coastlines-topo-150k.shp')

# Load shapefiles
urban_areas = gpd.read_file(urban_shapefile)
coastlines = gpd.read_file(coastline_shapefile)

output_coastline_shapefile = os.path.join(input_dir, 'urban', 'intersecting_coastline.shp')
# Apply a 100 meter buffer to the urban areas
buffered_urban_areas = urban_areas.buffer(100)

# Find the intersection of the buffered urban areas and the coastline
intersecting_coastlines = gpd.overlay(coastlines, gpd.GeoDataFrame(geometry=buffered_urban_areas), how='intersection')

# Save the intersecting coastline segments to a new shapefile
intersecting_coastlines.to_file(output_coastline_shapefile)

print(f"Intersecting coastline shapefile created at {output_coastline_shapefile}")

Intersecting coastline shapefile created at C:\Users\owyn\Documents\SURV509\coastal classification\inputs\urban\intersecting_coastline.shp


In [9]:
# Function to resample GeoJPEG to 0.3m resolution
def resample_to_30cm(file_path):
    output_file_path = file_path.replace('.jpg', '_resampled.jpg')
    gdal.Warp(output_file_path, file_path, xRes=0.3, yRes=0.3, resampleAlg='bilinear')
    return output_file_path

# Function to check intersection
def check_intersection(aerial_dir):
    valid_files = []
    for root, _, files in os.walk(aerial_dir):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.jpeg'):
                file_path = os.path.join(root, file)
                with rasterio.open(file_path) as src:
                    transform = src.transform
                    cell_width, cell_height = transform[0], -transform[4]

                    # Check if cell size is approximately 0.3m
                    if not (0.295 <= cell_width <= 0.305 and 0.295 <= cell_height <= 0.305):
                        file_path = resample_to_30cm(file_path)

                    with rasterio.open(file_path) as resampled_src:
                        bounds = resampled_src.bounds
                        geom = box(*bounds)
                        intersects_urban = urban_areas.intersects(geom).any()
                        intersects_coastline = coastlines.intersects(geom).any()
                    
                        if intersects_urban or intersects_coastline:
                            valid_files.append(file_path)
    return valid_files

def retile_images(geo_jpeg_paths, output_directory):
    """
    Retile the given list of GeoJPEG paths and save the output to the specified directory.
    
    Args:
        geo_jpeg_paths (list of str): List of paths to GeoJPEG files.
        output_directory (str): Directory where the retiled images will be saved.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Prepare the arguments for gdal_retile.py
    args = [
        'gdal_retile.py', 
        '-targetDir', output_directory, 
        '-ps', '224', '224',
        '-r', 'bilinear'
    ] + geo_jpeg_paths

    # Run the gdal_retile function
    sys.argv = args
    gdal_retile_main()

    print(f"Retiled images saved to {output_directory}")

def filter_tifs_by_coastline(retiled_dir, coastline_shapefile):
    """
    Filter TIFF files in the retiled directory, keeping only those that intersect with the coastline.
    
    Args:
        retiled_dir (str): Directory containing the retiled TIFF files.
        coastline_shapefile (str): Path to the coastline shapefile.
    """
    # Load coastline shapefile
    coastlines = gpd.read_file(coastline_shapefile)

    # List of TIFF files to keep
    valid_tifs = []

    # Iterate over TIFF files in the retiled directory
    for tif_file in glob.glob(os.path.join(retiled_dir, '*.tif')):
        with rasterio.open(tif_file) as src:
            bounds = src.bounds
            geom = box(*bounds)
            intersects_coastline = coastlines.intersects(geom).any()
            
            if intersects_coastline:
                valid_tifs.append(tif_file)
    
    # Move invalid TIFF files to a subfolder 'invalid'
    invalid_dir = os.path.join(retiled_dir, 'invalid')
    if not os.path.exists(invalid_dir):
        os.makedirs(invalid_dir)
    
    for tif_file in glob.glob(os.path.join(retiled_dir, '*.tif')):
        if tif_file not in valid_tifs:
            shutil.move(tif_file, invalid_dir)

    print(f"Filtered TIFF files saved in {retiled_dir}. Invalid files moved to {invalid_dir}")

def filter_tifs_by_dimensions(retiled_dir, target_width=224, target_height=224):
    """
    Filter TIFF files in the retiled directory, moving those that do not match the target dimensions to an 'invalid' folder.
    
    Args:
        retiled_dir (str): Directory containing the retiled TIFF files.
        target_width (int): Target width of the TIFF files.
        target_height (int): Target height of the TIFF files.
    """
    # Directory for invalid TIFF files
    invalid_dir = os.path.join(retiled_dir, 'invalid')
    if not os.path.exists(invalid_dir):
        os.makedirs(invalid_dir)
    
    # Iterate over TIFF files in the retiled directory
    for tif_file in glob.glob(os.path.join(retiled_dir, '*.tif')):
        with rasterio.open(tif_file) as src:
            width, height = src.width, src.height
            
        if width != target_width or height != target_height:
            shutil.move(tif_file, invalid_dir)
            print(f"Moved {tif_file} to {invalid_dir} due to incorrect dimensions ({width}x{height})")
    
    print(f"Checked TIFF files for dimensions in {retiled_dir}. Invalid files moved to {invalid_dir}")


In [17]:
# Main processing loop

# Dictionary to store valid JPEG paths
valid_jpegs = {}

aerial_imagery_base_dir = os.path.join(input_dir, '0.3m Aerial Imagery')

for region_name in os.listdir(aerial_imagery_base_dir):
    region_dir = os.path.join(aerial_imagery_base_dir, region_name)
    if os.path.isdir(region_dir):
        retile_output_dir = os.path.join(region_dir, 'retiled')
        
        if not os.path.exists(retile_output_dir) or not os.listdir(retile_output_dir):
            valid_jpegs[region_name] = check_intersection(region_dir)
            print(f"Intersecting tiles found for {region_name}")
            
            # Retile the valid files
            retile_images(valid_jpegs[region_name], retile_output_dir)
        
        # Filter TIFF files by intersection with coastline (urban coastline)
        filter_tifs_by_coastline(retile_output_dir, output_coastline_shapefile)
        filter_tifs_by_dimensions(retile_output_dir, target_width=224, target_height=224)

print("Processing complete.")

Filtered TIFF files saved in C:\Users\owyn\Documents\SURV509\coastal classification\inputs\0.3m Aerial Imagery\christchurch\retiled. Invalid files moved to C:\Users\owyn\Documents\SURV509\coastal classification\inputs\0.3m Aerial Imagery\christchurch\retiled\invalid
Checked TIFF files for dimensions in C:\Users\owyn\Documents\SURV509\coastal classification\inputs\0.3m Aerial Imagery\christchurch\retiled. Invalid files moved to C:\Users\owyn\Documents\SURV509\coastal classification\inputs\0.3m Aerial Imagery\christchurch\retiled\invalid
Filtered TIFF files saved in C:\Users\owyn\Documents\SURV509\coastal classification\inputs\0.3m Aerial Imagery\dunedin\retiled. Invalid files moved to C:\Users\owyn\Documents\SURV509\coastal classification\inputs\0.3m Aerial Imagery\dunedin\retiled\invalid
Checked TIFF files for dimensions in C:\Users\owyn\Documents\SURV509\coastal classification\inputs\0.3m Aerial Imagery\dunedin\retiled. Invalid files moved to C:\Users\owyn\Documents\SURV509\coastal cl

In [18]:
total_valid_tifs = 0

for region_name in os.listdir(aerial_imagery_base_dir):
    region_dir = os.path.join(aerial_imagery_base_dir, region_name)
    if os.path.isdir(region_dir):
        retile_output_dir = os.path.join(region_dir, 'retiled')
        if os.path.exists(retile_output_dir):
            valid_tifs = glob.glob(os.path.join(retile_output_dir, '*.tif'))
            total_valid_tifs += len(valid_tifs)

print(f"Total number of valid TIFF files: {total_valid_tifs}")

Total number of valid TIFF files: 4502


### Labelling data (Christchurch and Dunedin)
Labelling coastline for Christchurch and Dunedin

In [21]:
labelled_areas_shapefile = os.path.join(input_dir, 'training', 'labelled areas.shp')
output_built_up_coastline_shapefile = os.path.join(input_dir, 'training', 'built_up_coastline.shp')

labelled_areas = gpd.read_file(labelled_areas_shapefile)
# Find the intersection of the intersecting coastline with the labelled areas
built_up_coastline = gpd.overlay(intersecting_coastlines, labelled_areas, how='intersection')

# Save the resulting segments to a new shapefile
built_up_coastline.to_file(output_built_up_coastline_shapefile)

print(f"Built-up coastline shapefile created at {output_built_up_coastline_shapefile}")

Built-up coastline shapefile created at C:\Users\owyn\Documents\SURV509\coastal classification\inputs\training\built_up_coastline.shp


### Creating the label dataset

In [24]:
# List of regions to process
regions_to_process = ['christchurch', 'dunedin']

# Initialise list to store TIFF file names and their labels
tif_labels = []

# Iterate through specified regions
for region_name in regions_to_process:
    region_dir = os.path.join(aerial_imagery_base_dir, region_name, 'retiled')
    if os.path.isdir(region_dir):
        valid_tif_files = glob.glob(os.path.join(region_dir, '*.tif'))
        
        for tif_file in valid_tif_files:
            with rasterio.open(tif_file) as src:
                bounds = src.bounds
                geom = box(*bounds)
                intersects_built_up = built_up_coastline.intersects(geom).any()
                label = 1 if intersects_built_up else 0
                tif_labels.append((tif_file, label))

# Save the labels to a CSV file
tif_labels_df = pd.DataFrame(tif_labels, columns=['tif_file', 'label'])
output_csv = os.path.join(input_dir, 'training', 'tif_labels.csv')
tif_labels_df.to_csv(output_csv, index=False)

# Summary
total_observations = len(tif_labels)
total_ones = sum(label for _, label in tif_labels)
total_zeros = total_observations - total_ones

print(f"Total number of observations: {total_observations}")
print(f"Number of 1s: {total_ones}")
print(f"Number of 0s: {total_zeros}")
print(f"Labels saved to {output_csv}")

Total number of observations: 2482
Number of 1s: 1075
Number of 0s: 1407
Labels saved to C:\Users\owyn\Documents\SURV509\coastal classification\inputs\training\tif_labels.csv
